<h2> Automatic filtering </h2> 

In [1]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dpp

2023-09-15 00:39:58 Didis-MacBook-Pro.local metapub.config[27774] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
import os
import numpy as np
import string
import math
from nltk import ngrams
import PyPDF2
import time

<h3> Predefined fucntions: </h3> 

In [3]:
def info_filling(input_path, output_path, start, end):
    # scan each row in the potential related literature and extract information
    df = pd.read_csv(input_path, header=None, sep=",")
    df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

    for ind in range(start, end):
        # info = {
        #     "doi": np.nan,
        #     "pmid": np.nan,
        #     "pmcid": np.nan,
        #     "title": np.nan,
        #     "abstract": np.nan,
        #     "keywords": np.nan,
        #     "pdf_link": np.nan
        # }

        # initialzie
        index = df.at[ind, "INDEX"]
        doi = df.at[ind, "DOI"]
        pmid = df.at[ind, "PMID"]
        pmcid = df.at[ind, "PMCID"]
        full_text_url = df.at[ind, "FULL_TEXT_URL"]
        full_text_source = df.at[ind, "FULL_TEXT_SOURCE"]
        pdf_url = np.nan
        pdf_source = np.nan
        title = df.at[ind, "TITLE"]
        abstract = df.at[ind, "ABSTRACT"]
        keywords = df.at[ind, "KEYWORDS"]

        if full_text_url != full_text_url: # full text url not found
            if df.at[ind, "PDF_URL"] == df.at[ind, "PDF_URL"]:
                url = str(df.at[ind, "PDF_URL"]).strip()
                url1, status_code = plib.get_final_redirected_url(url)
                if status_code == 200:
                    pdf_url = url
                    pdf_source = pdf_url.split("://")[1].split("/")[0]
                elif status_code == 403:
                    # print(status_code, "when getting final redirected url: ", url)
                    pdf_url = url
                    pdf_source = pdf_url.split("://")[1].split("/")[0]
                else:
                    print(status_code, "when getting final redirected url: ", url)
                    pdf_url = np.nan
                    pdf_source = np.nan  
            else:
                print("full text url and pdf url are not available!")
                pdf_url = np.nan
                pdf_source = np.nan
        elif ('.pdf' in full_text_url and full_text_url.split('.pdf')[1] == '') or ('.PDF' in full_text_url and full_text_url.split('.PDF')[1] == ''):
            print("full text url is a pdf file: ", full_text_url)
            pdf_url = full_text_url
            pdf_source = full_text_source
            full_text_url = np.nan
            full_text_source = np.nan
        else: # full text url found
            flag = False
            for website in params.websites:
                if website in full_text_source:
                    flag = True
                    break
            if not flag:
                continue

            url = str(full_text_url).strip()
            try:
                info = extract_info.extract_info_from_webpage(url, params.websites)
            except:
                raise Exception("Error! Cannot extract information from the webpage: ", url)
            
            # doi
            if info['doi'] == info['doi'] and doi == doi and info['doi'] != doi:
                print(doi)
                print(info['doi'])

            if info['doi'] == info['doi']:
                doi = info['doi'].lower()
            else:
                doi = doi
            
            # pmid
            if info['pmid'] == info['pmid'] and df.at[ind, "PMID"] == df.at[ind, "PMID"] and str(int(info['pmid'])) != str(int(df.at[ind, "PMID"])):
                print(str(int(df.at[ind, "PMID"]))) 
                print(str(int(info['pmid'])))      

            if info['pmid'] == info['pmid']:
                pmid = str(int(info['pmid']))
            elif pmid == pmid:
                pmid = str(int(pmid)).strip()
            else:
                pmid = np.nan
            
            # pmcid
            if info['pmcid'] == info['pmcid'] and pmcid == pmcid and info['pmcid'] != pmcid:
                print(pmcid)
                print(info['pmcid'])

            if info['pmcid'] == info['pmcid']:
                pmcid = info['pmcid']
            else:
                pmcid = df.at[ind, "PMCID"]
            
            # full_text_url, full_text_surce
            if full_text_url == full_text_url:
                full_text_source = full_text_url.split("://")[1].split("/")[0]
            else:
                print("full text url is not available")
                full_text_url = np.nan
                full_text_source = np.nan

            if pdf_url != pdf_url and info['pdf_link'] == info['pdf_link']:
                pdf_url = str(info['pdf_link']).strip()
                pdf_source = pdf_url.split("://")[1].split("/")[0]
                # try:
                #     pdf_url, status_code = plib.get_final_redirected_url(url)
                #     if status_code == 200:
                #         pdf_source = pdf_url.split("://")[1].split("/")[0]
                #     elif status_code == 403:
                #         # print(status_code, "when getting final redirected url: ", url)
                #         pdf_source = pdf_url.split("://")[1].split("/")[0]
                #     else:
                #         print(status_code, "when getting final redirected url: ", url)
                #         pdf_url = np.nan
                #         pdf_source = np.nan
                # except:
                #     pdf_url = np.nan
                #     pdf_source = np.nan 
                    
            if pdf_url != pdf_url and df.at[ind, "PDF_URL"] == df.at[ind, "PDF_URL"]:
                print("PDF_URL not extracted from info: , but existed already: ", df.at[ind, "PDF_URL"])
                pdf_url = df.at[ind, "PDF_URL"]
                pdf_source = pdf_url.split("://")[1].split("/")[0]
            
            if pdf_url != pdf_url:
                if full_text_url == full_text_url and full_text_url.split("://")[1].split("/")[0] == 'www.ncbi.nlm.nih.gov':
                    if doi == doi:
                        url = "https://doi.org/" + doi
                        url, status_code = plib.get_final_redirected_url(url)
                        info = extract_info.extract_info_from_webpage(url, params.websites)
                        full_text_url = url
                        full_text_source = url.split("://")[1].split("/")[0]
                        pdf_url = info["pdf_link"]
                        pdf_source = pdf_url.split("://")[1].split("/")[0]
                    else:
                        pdf_url = np.nan
                        pdf_source = np.nan
                else:  
                    print("PDF_URL not found for: ", doi, pmid, pmcid, full_text_url)
                    pdf_url = np.nan
                    pdf_source = np.nan
                
            # title
            if info['title'] == info['title']:
                title = info['title']
                title = title.replace(";", ",")
            else:
                title = title
            
            # abstract
            if info['abstract'] == info['abstract']:
                abstract = info['abstract']
                abstract = ''.join(e for e in abstract if (e.isalpha() or e == " " or e == "-"))
            else:
                abstract = abstract
            
            # keywords
            if info['keywords'] == info['keywords']:
                keywords = info['keywords']
                keywords = keywords.replace(";", ",")
                keywords = ''.join(e for e in keywords if (e.isalpha() or e == " " or e == "-" or e == ","))
            else:
                keywords = keywords
        
        columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
        
        row = {
            "INDEX": [index],
            "DOI": [doi],
            "PMID": [pmid],
            "PMCID": [pmcid],
            "FULL_TEXT_URL": [full_text_url],
            "FULL_TEXT_SOURCE": [full_text_source],
            "PDF_URL": [pdf_url],
            "PDF_SOURCE": [pdf_source],
            "TITLE": [title],
            "ABSTRACT": [abstract],
            "KEYWORDS": [keywords]
        }
        # print(row)

        if not plib.add_row_to_csv(output_path, row, columns):
            print("Error detected when adding a row to csv!")
        
        print(ind)
# --------------------start of test code--------------------
# input_path = fpath.poten_litera_ids_ftl_filled
# output_path = fpath.poten_litera_litera_db

# # clear file
# plib.clear_file(output_path)

# info_filling(input_path, output_path)
# ---------------------end of test code---------------------

In [4]:
def download_pdf(input_path, pdf_folder, start, end):
    df = pd.read_csv(input_path, header=None, sep=',')
    df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

    for ind in range(start, end):
        time.sleep(3)
        
        # the flag to indicate whether the pdf is downloaded successfully, if not, save the row to poten_litera_pdf_not_available.csv
        flag = False

        pdf_url = df.at[ind, "PDF_URL"]

        # pdf_url not found
        if pdf_url != pdf_url:
            flag = False
        # pdf_url found
        else:
            doi = df.at[ind, "DOI"]
            if dpp.download_and_rename_pdf(pdf_url, doi, df.at[ind, "INDEX"], pdf_folder):
                flag = True
            else:
                print(df.at[ind, "FULL_TEXT_URL"])
                flag = False
        
        if not flag:
            print("PDF_URL not found or PDF not successfully downloaded for: ")
            print("\n")
            print(df.at[ind, "INDEX"], df.at[ind, "DOI"], df.at[ind, "PMID"], df.at[ind, "PMCID"], df.at[ind, "FULL_TEXT_URL"], df.at[ind, "FULL_TEXT_SOURCE"], df.at[ind, "PDF_URL"], df.at[ind, "PDF_SOURCE"], df.at[ind, "TITLE"], df.at[ind, "ABSTRACT"], df.at[ind, "KEYWORDS"])
            # print("\n")

        #     # save the row to poten_litera_pdf_not_available.csv
        #     columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
        #     row = {
        #         "INDEX": [df.at[ind, "INDEX"]],
        #         "DOI": [df.at[ind, "DOI"]],
        #         "PMID": [df.at[ind, "PMID"]],
        #         "PMCID": [df.at[ind, "PMCID"]],
        #         "FULL_TEXT_URL": [df.at[ind, "FULL_TEXT_URL"]],
        #         "FULL_TEXT_SOURCE": [df.at[ind, "FULL_TEXT_SOURCE"]],
        #         "PDF_URL": [df.at[ind, "PDF_URL"]],
        #         "PDF_SOURCE": [df.at[ind, "PDF_SOURCE"]],
        #         "TITLE": [df.at[ind, "TITLE"]],
        #         "ABSTRACT": [df.at[ind, "ABSTRACT"]],
        #         "KEYWORDS": [df.at[ind, "KEYWORDS"]]
        #     }
        #     if not plib.add_row_to_csv(poten_litera_pdf_not_available, row, columns):
        #         print("Error detected when adding a row to csv!")
        #     # print(row)

        line_number_in_csv = ind + 1
        print(ind, " Line number:", line_number_in_csv, " INDEX:", int(df.at[ind, "INDEX"]))
        print("\n")

In [5]:
def pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract):
    text_path = pdf_path.split(".pdf")[0].split("/")[1] + ".txt"
    
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(start_page, page_max + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())
            if text.split(" ") > text_length_to_extract:
                break

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [6]:
def count_keyword(text, keyword, keyword_length):
    # remove non-alphabetic characters but keep spaces and "-"
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    # print(text)
    text = text.strip().lower()
    # print(text)
    
    words = []
    # sentence = 'I have a laptop case and a laptop bag'
    n = keyword_length
    n_grams = ngrams(text.split(), n)
    for gram in n_grams:
        word = gram[0]
        if n > 0:
            for i in range(1, n):
                word = word + " " + gram[i]
        words.append(word)
    
    # print(words)
    
    word_count = 0
    for word in words:
        # print(word)
        if word == keyword:
            word_count += 1
    return word_count
# --------------------start of test code--------------------
# text = 'This apple 6i7s very tasty？、  2but th&e banana this is not delicious at Is all.6'
# keyword = 'this apple'
# count = count_keyword(text, keyword, 2)
# print(count)
# ---------------------end of test code---------------------

In [7]:
def count_freq_from_liter(text, on_topic_kws, type):
    text = ''.join(e for e in text if (e.isalpha() or e == " " or e == "-"))
    # print(text)
    text = text.strip().lower()
    # print(text)

    text_length = len(text.split(" "))
    keywords_count = dict()
    keywords_fre = dict()

    # count the on-topic keywords
    for i in range(len(on_topic_kws)):
        word_count = count_keyword(text, on_topic_kws[i], len(on_topic_kws[i].split(" ")))
        print(word_count)
        if type == "count":
            keywords_count[on_topic_kws[i]] = word_count
        elif type == "frequency":
            keywords_fre[on_topic_kws[i]] = math.ceil((word_count*100/text_length))/100
        else:
            raise Exception("Error! The only two options for type are 'count' or 'frequency'!")
    
    if type == "count":
        return keywords_count
    elif type == "frequency":
        return keywords_fre
    else:
        raise Exception("Error! The only two options for type are 'count' or 'frequency'!")
# --------------------start of test code--------------------
# text = 'Vision for action: thalamic and cortical inputs to the macaque superior tract neural tracing, parietal lobule The dorsal visual stream, the cortical circuit that in the primate brain is mainly dedicated to the visual control of actions, is split into two routes, a lateral and a medial one, both involved in coding different aspects of sensorimotor control of actions. The lateral route, named "lateral grasping network", is mainly involved in the control of the distal part of prehension, namely grasping and manipulation. The medial route, named "reach-to-grasp network", is involved in the control of the full deployment of prehension act, from the direction of arm movement to the shaping of the hand according to the object to be grasped. In macaque monkeys, the reach-to-grasp network (the target of this review) includes areas of the superior parietal lobule (SPL) that hosts visual and somatosensory neurons well suited to control goal-directed limb movements toward stationary as well as moving objects. After a brief summary of the neuronal functional properties of these areas, we will analyze their cortical and thalamic inputs thanks to retrograde neuronal tracers separately injected into the SPL areas V6, V6A, PEc, and PE. These areas receive visual and somatosensory information distributed in a caudorostral, visuosomatic trend, and some of them are directly connected with the dorsal premotor cortex. This review is particularly focused on the origin and type of visual information reaching the SPL, and on the functional role this information can play in guiding limb interaction with objects in structured and dynamic environments. Area PEc; Area V6; Area V6A; Dorsal visual stream; Goal-directed arm movement; Sensorimotor integration.'
# keywords_count_fre = count_freq_from_liter(text, params.on_topic_kws, type="count")
# print(keywords_count_fre)
# ---------------------end of test code---------------------

In [8]:
def calcul_index(keywords_count_or_fre, on_topic_kws_weights):
    index = 0
    for key in keywords_count_or_fre.keys():
        index += keywords_count_or_fre[key] * on_topic_kws_weights[key]
    return index
# --------------------start of test code--------------------
# keywords_count_or_fre = {}
# index = calcul_related(keywords_count_or_fre, params.on_topic_kws_weights)
# print(index)
# ---------------------end of test code---------------------

In [9]:
def weight_and_rank(input_path, output_path, on_topic_kws_weights):
    df = pd.read_csv(input_path, header=None, sep=",")
    df.columns = ["DOI", "PMID", "PMCID", "full_text_link", "full_text_source", "pdf_link", "Title", "Abstract", "Keywords", "index"]

    for ind in df.index:
        text = ""
        if df.at[ind, "Title"] == df.at[ind, "Title"]:
            text += df.at[ind, "Title"] + " "
        if df.at[ind, "Abstract"] == df.at[ind, "Abstract"]:
            text += df.at[ind, "Abstract"] + " "
        if df.at[ind, "Keywords"] == df.at[ind, "Keywords"]:
            text += df.at[ind, "Keywords"] + " "
        # print(text)
        # type = "count"
        type = "frequency"
        keywords_count_or_fre = count_freq_from_liter(text, params.on_topic_kws, type)
        index = calcul_index(keywords_count_or_fre, on_topic_kws_weights)
        # print(index)
        df.at[ind, "index"] = index
        # print(ind)
    
    # rank
    df = df.sort_values(by=["index"], ascending=False)
    df.to_csv(output_path, header=True, index=False)
    print("Weighting and ranking the potentially related literature succeded!")
    print("Enjoy reading!")
# --------------------start of test code--------------------
# test code
# ---------------------end of test code---------------------

<h3> Main program: </h3> 

In [10]:
# # step 1: extract and filling info
# input_path = fpath.poten_litera_ids_ftl_filled_filtered
# output_path = fpath.poten_litera_litera_db

# # clear file
# # plib.clear_file(output_path)

# info_filling(input_path, output_path, 0, 10980)

In [11]:
# # process the resuls: remove duplicates and reset index
# input_path = fpath.poten_litera_litera_db
# output_path = fpath.poten_litera_litera_db

# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

# identifiers = ["INDEX", "DOI", "PMID", "PMCID"]

# for identifier in identifiers:
#     remove_dup_by = identifier
#     df = df[df[remove_dup_by].isnull() | ~df[df[remove_dup_by].notnull()].duplicated(subset=remove_dup_by, keep='last')]

# # reset index
# df.reset_index(drop=True, inplace=True)

# df.to_csv(output_path, header=False, index=False)
# print("Duplication removed.")

# input_path = fpath.poten_litera_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# print(df.shape)
# # (10776, 11)

In [12]:
# # list all pdf source
# input_path = fpath.poten_litera_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
# pdf_source_set = set(df['PDF_SOURCE'].tolist())
# print(pdf_source_set)
# # {'www.ahajournals.org', 'anatomypubs.onlinelibrary.wiley.com', 'citeseerx.ist.psu.edu', 'www.nature.com', 
# # 'iovs.arvojournals.org', 'www.microbiologyresearch.org', 'nyaspubs.onlinelibrary.wiley.com', 'ahuman.org', 
# # 'karger.com', 'www.imrpress.com', 'www.researchsquare.com', 'link.springer.com', 'www.ijpp.com', 
# # 'europepmc.org', nan, 'www.cell.com', 'www.bu.edu', 'www.ncbi.nlm.nih.gov', 'jamanetwork.com', 
# # 'www.thieme-connect.de', 'www.science.org', 'physoc.onlinelibrary.wiley.com', 'deepblue.lib.umich.edu', 
# # 'bpb-us-e1.wpmucdn.com', 'www.researchgate.net', 'ieeexplore.ieee.org', 'zsp.com.pk', 'journals.biologists.com', 
# # 'journals.aps.org', 'papers.ssrn.com', 'academic.oup.com', 'onlinelibrary.wiley.com', 'www.hifo.uzh.ch', 
# # 'royalsocietypublishing.org', 'www.biorxiv.org', 'www.ingentaconnect.com', 'ujms.net', 'enpubs.faculty.ucdavis.edu', 
# # 'ajp.psychiatryonline.org', 'n.neurology.org', 'www.annualreviews.org', 'ruor.uottawa.ca', 'neuro.psychiatryonline.org', 
# # 'www.jstage.jst.go.jp', 'synapse.koreamed.org', 'journals.physiology.org', 'linkinghub.elsevier.com', 
# # 'www.tandfonline.com', 'www.jneurosci.org', 'analyticalsciencejournals.onlinelibrary.wiley.com', 'pubs.asahq.org', 
# # 'thejns.org', 'biomedical-engineering-online.biomedcentral.com', 'journals.sagepub.com', 'direct.mit.edu', 
# # 'pubs.acs.org', 'pharmrev.aspetjournals.org', 'journals.lww.com', 'jnm.snmjournals.org', 'jpet.aspetjournals.org', 
# # 'movementdisorders.onlinelibrary.wiley.com'}

In [13]:
# pdf_source_set = [
#     'www.ahajournals.org', 'wiley.com', 'citeseerx.ist.psu.edu', 'www.nature.com', 
#     'iovs.arvojournals.org', 'www.microbiologyresearch.org', 'ahuman.org', 
#     'karger.com', 'www.imrpress.com', 'www.researchsquare.com', 'link.springer.com', 'www.ijpp.com', 
#     'europepmc.org', 'www.cell.com', 'www.bu.edu', 'www.ncbi.nlm.nih.gov', 'jamanetwork.com', 
#     'www.thieme-connect.de', 'www.science.org', 'deepblue.lib.umich.edu', 
#     'bpb-us-e1.wpmucdn.com', 'www.researchgate.net', 'ieeexplore.ieee.org', 'zsp.com.pk', 'journals.biologists.com', 
#     'journals.aps.org', 'papers.ssrn.com', 'academic.oup.com', 'www.hifo.uzh.ch', 
#     'royalsocietypublishing.org', 'www.biorxiv.org', 'www.ingentaconnect.com', 'ujms.net', 'enpubs.faculty.ucdavis.edu', 
#     'psychiatryonline.org', 'n.neurology.org', 'www.annualreviews.org', 'ruor.uottawa.ca', 
#     'www.jstage.jst.go.jp', 'synapse.koreamed.org', 'journals.physiology.org', 'linkinghub.elsevier.com', 
#     'www.tandfonline.com', 'www.jneurosci.org', 'pubs.asahq.org', 
#     'thejns.org', 'biomedcentral.com', 'journals.sagepub.com', 'direct.mit.edu', 
#     'pubs.acs.org', 'aspetjournals.org', 'journals.lww.com', 'jnm.snmjournals.org'
# ]

# input_path = fpath.poten_litera_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# df.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
# for website in pdf_source_set:
#     for ind in df.index:
#         if df.at[ind, "PDF_SOURCE"] != df.at[ind, "PDF_SOURCE"]:
#             continue
#         if website in df.at[ind, "PDF_SOURCE"]:
#             print("# " + website)
#             print("\"" + df.at[ind, "PDF_URL"] + "\"")
#             break

In [14]:
# input_path = fpath.poten_litera_litera_db
# df = pd.read_csv(input_path, header=None, sep=',')
# print(df.shape)
# print(df.head(5))
# # (10776, 11)

In [15]:
# step 2: download pdfs and rename them to build a database
input_path = fpath.poten_litera_db
pdf_folder = fpath.pdf_folder
download_pdf(input_path, pdf_folder, 2827, 10776)

Failed downloading PDF: 1830 https://neuro.psychiatryonline.org/doi/reader/10.1176/jnp.8.2.125
https://neuro.psychiatryonline.org/doi/abs/10.1176/jnp.8.2.125
PDF_URL not found or PDF not successfully downloaded for: 


1830 10.1176/jnp.8.2.125 9081547.0 nan https://neuro.psychiatryonline.org/doi/abs/10.1176/jnp.8.2.125 neuro.psychiatryonline.org https://neuro.psychiatryonline.org/doi/reader/10.1176/jnp.8.2.125 neuro.psychiatryonline.org Anatomy and function of the orbital frontal cortex, I: anatomy, neurocircuitry, and obsessive-compulsive disorder Many neuroimaging studies have implicated the orbital frontal cortex OFC in the pathophysiology of obsessive-compulsive disorder In recent years there have been significant advances in elucidating the anatomical characteristics of the OFC in nonhuman primates The authors review literature on the cytoarchitecture and afferent and efferent connections of the OFC giving particular attention to the OFCs relationship to limbic and paralimbic regi

WebDriverException: Message: Process unexpectedly closed with status 0


In [ ]:
# # record the literatures whose pdfs are not available
# poten_litera_pdf_not_available = fpath.poten_litera_pdf_not_available
# # plib.clear_file(poten_litera_pdf_not_available)

In [ ]:
# # # process, correct the INDEX in potential_related_literature_testing_set_300_read.csv
# # input_path = fpath.poten_litera_testing_set_300_read
# # output_path = fpath.poten_litera_testing_set_300_read_index_corrected
# # plib.clear_file(output_path)
# # db_path = fpath.poten_litera_litera_db

# # df_input = pd.read_csv(input_path, header=0, sep=',')
# # df_input.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS", "RELEVANCE"]
# # df_db = pd.read_csv(db_path, header=None, sep=',')
# # df_db.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
# # df_db = df_db.fillna(0)
# # df_db = df_db.astype({"PMID": int})
# # # print(df_input.shape)
# # # print(df_input.head(5))
# # # (300, 12)
# # # print(df_db.shape)
# # # print(df_db.head(10))
# # # (10776, 11)

# # for ind in df_input.index:
# #     index = df_input.at[ind, "INDEX"]
# #     doi = df_input.at[ind, "DOI"]
# #     pmid = df_input.at[ind, "PMID"]
# #     # print(pmid, df_db.at[ind, "PMID"])
# #     # print(pmid.type(), df_db.at[ind, "PMID"].type())
# #     pmcid = df_input.at[ind, "PMCID"]
# #     full_text_url = df_input.at[ind, "FULL_TEXT_URL"]
# #     full_text_source = df_input.at[ind, "FULL_TEXT_SOURCE"]
# #     title = df_input.at[ind, "TITLE"].lower()

# #     if doi == doi:
# #         try:
# #             index = df_db.loc[df_db["DOI"] == doi, 'INDEX'].values[0]
# #             df_input.at[ind, "INDEX"] = index
# #         except:
# #             print("DOI not found in db:", df_input.at[ind, "INDEX"], df_input.at[ind, "RELEVANCE"])
# #             df_input.drop(ind, inplace=True)
# #     elif pmid == pmid:
# #         try:
# #             index = df_db.loc[df_db["PMID"]==int(pmid), 'INDEX'].values[0]
# #             df_input.at[ind, "INDEX"] = index
# #         except:
# #             print("PMID not found in db:", df_input.at[ind, "INDEX"], df_input.at[ind, "RELEVANCE"])
# #             df_input.drop(ind, inplace=True)
# #     elif pmcid == pmcid:
# #         index = df_db.loc[df_db["PMCID"] == pmcid, 'INDEX'].values[0]
# #         df_input.at[ind, "INDEX"] = index
# #     elif title.lower() == title.lower():
# #         index = df_db.loc[df_db["TITLE"].str.lower() == title, 'INDEX'].values[0]
# #         df_input.at[ind, "INDEX"] = index
# #     else:
# #         print("ALL 4 identifiers and title are missing:", df_input.at[ind, "INDEX"], df_input.at[ind, "RELEVANCE"])
    
# # df_input.reset_index(drop=True, inplace=True)
# # df_input.to_csv(output_path, header=True, index=False)

# corrected = fpath.poten_litera_testing_set_300_read_index_corrected
# df_input = pd.read_csv(corrected, header=0, sep=',')
# print(df_input.shape)
# # (292, 12)
# print(df_input.head(5))

In [ ]:
# # test if the result matches the db
# result_path = fpath.poten_litera_testing_set_300_read_index_corrected
# db_path = fpath.poten_litera_litera_db

# df_result= pd.read_csv(result_path, header=0, sep=',')
# df_result.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS", "RELEVANCE"]
# df_db = pd.read_csv(db_path, header=None, sep=',')
# df_db.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]
# # print(df_result.shape)
# # print(df_result.head(5))
# # (292, 12)
# # print(df_db.shape)
# # print(df_db.head(10))
# # (10776, 11)

# for ind in df_result.index:
#     index = int(df_result.at[ind, "INDEX"])
#     title = ''.join([char for char in df_result.at[ind, "TITLE"].lower() if re.match(r'[a-z\s]', char)])
#     cleaned_title = re.sub(r'\s+', ' ', title).strip()
#     title_db = ''.join([char for char in df_db.loc[df_db["INDEX"].astype(int) == index, 'TITLE'].values[0].lower() if re.match(r'[a-z\s]', char)])
#     cleaned_title_db = re.sub(r'\s+', ' ', title_db).strip()
    
#     if cleaned_title == cleaned_title_db:
#         pass
#     else:
#         # pass
#         print(index)
#         print(cleaned_title)
#         print(cleaned_title_db)
#         print("\n")

In [ ]:
# def download_pdf_300(db_path, test_path, pdf_folder, start, end):
#     df_db = pd.read_csv(db_path, header=None, sep=',')
#     df_db.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS"]

#     df_test = pd.read_csv(test_path, header=0, sep=',')
#     df_test.columns = ["INDEX", "DOI", "PMID", "PMCID", "FULL_TEXT_URL", "FULL_TEXT_SOURCE", "PDF_URL", "PDF_SOURCE", "TITLE", "ABSTRACT", "KEYWORDS", "RELEVANT"]

#     for ind in range(start, end):
#         # time.sleep(2)
#         flag = False
#         index = df_test.at[ind, "INDEX"]
#         pdf_url = df_db.loc[df_db['INDEX'] == index, "PDF_URL"].values[0]
        
#         # pdf_url found
#         if pdf_url != pdf_url:
#             flag = False
#         else:
#             doi = df_test.at[ind, "DOI"]
#             if doi == doi:
#                 doi = doi.lower().strip()
#             else:
#                 pass
#             if dpp.download_and_rename_pdf(pdf_url, doi, index, pdf_folder):
#                 flag = True
#             else:
#                 flag = False
        
#         if not flag:
#             print("\n")
#             print("PDF_URL not found or PDF not successfully downloaded for:")
#             print(index)
#             print(df_db.loc[df_db['INDEX'] == index, "FULL_TEXT_URL"].values[0])
#             print(df_db.loc[df_db['INDEX'] == index, "PDF_URL"].values[0])
#             print(df_db.loc[df_db['INDEX'] == index, "TITLE"].values[0])

#         line_number_in_csv = ind + 1
#         print(ind, " Line number:", line_number_in_csv, " INDEX:", index)
#         print("\n")

# # # download the pdfs of 300 test papers
# db_path = fpath.poten_litera_db
# test_path = fpath.poten_litera_testing_set_300_read_index_corrected
# pdf_folder = fpath.pdf_folder
# download_pdf_300(db_path, test_path, pdf_folder, 0, 300)

In [ ]:
# # # step 3: process the pdf and extract text to store to a text file
# pdf_path = fpath.pdf_folder
# text_path = fpath.text_folder
# start_page = 0
# end_page = 1
# text_length_to_extract = 2000
# pdf2text(pdf_path, text_path, start_page, end_page, text_length_to_extract)

# from pathlib import Path
# pathlist = Path(pdf_path).glob('**/*.pdf')
# for path in pathlist:
#      # because path is object not string
#      path_in_str = str(path)
#      print(path_in_str)
#      pdf2text(path_in_str, text_path, start_page, end_page, text_length_to_extract)

In [ ]:
# # step 3: fill in columns containing information of the number of times keywords appear and assign index to each literature and rank them
# input_path = fpath.poten_litera_litera_db
# test_path = fpath.poten_litera_testing_set_300_read
# output_path = fpath.poten_litera_litera_db_ranked

# # clear file
# plib.clear_file(output_path)

# weight_and_rank(input_path, output_path, on_topic_kws_weights)

<h3> Next step: manually read papers and find all actually related literature </h3>